In [3]:
import torch
import numpy as np
import os
import time
from torch.utils.data import Dataset, DataLoader
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.omni_tools import copy2cpu as c2c
from human_body_prior.tools.rotation_tools import aa2matrot,matrot2aa,local2global_pose
from utils import utils_transform
import glob
from IPython import embed
import time
import copy
import pickle
from models.network import AvatarPoser as net

In [4]:
#load pretrained model
pretrained_model = torch.load('./model_zoo/avatarposer.pth')

support_dir = 'support_data/'
bm_fname_male = os.path.join(support_dir, 'body_models/smplh/{}/model.npz'.format('male'))
dmpl_fname_male = os.path.join(support_dir, 'body_models/dmpls/{}/model.npz'.format('male'))
num_betas = 16 # number of body parameters
num_dmpls = 8 # number of DMPL parameters
device = torch.device('cuda')
body_model = BodyModel(bm_fname=bm_fname_male, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname_male)

model = net(input_dim=54,
            output_dim=132,
            num_layer=3,
            embed_dim=256,
            nhead = 8,
            body_model = body_model,
            device = device
        )

In [5]:
##########################################################################################

# VR Input Format :
# Write2DArrToZip_npy(zip, "controller_rot_global.npy", controller_rot_global);
# Write2DArrToZip_npy(zip, "controller_trans_global.npy", controller_trans_global);
# Write2DArrToZip_npy(zip, "p_a.npy", p_a);
# Write2DArrToZip_npy(zip, "R_a.npy", R_a);
# Write2DArrToZip_npy(zip, "M.npy", M);
# Write2DArrToZip_npy(zip, "aw.npy", aw);

##########################################################################################

# controller_rot_global.npy: (frame, 3*3*3)
# controller_trans_global.npy: (frame, 3*3)
# 나머지도 전부 (length, -1) 꼴로 되어있다.

##########################################################################################

# # Anyway, we need unity_pose_rmat.npy and unity_trans.npy
# # Or, avatar_local_pose_rmat.npy, avatar_position_global.npy, P_a.npy, R_a.npy, M.npy, aw.npy

# unity coordinates 
# #unity_poses_aa = unity_poses_aa.reshape(length, -1),       # length x 22 x 3  
# unity_trans = unity_trans.reshape(length, -1),              # length x 3             , pelvis_translation
# unity_pose_rmat = unity_pose_rmat.reshape(length, -1),      # length x 22 x 3 x 3    , local_rotations (all)

# world coordinates
# world_pose_rotmat = world_rotmat.reshape(length, -1),       # length x 22 x 3 x 3     , world_rotation (all)
# world_pose_transform = world_transform.reshape(length, -1), # length x 22 x 3         , world_translation (all)

# avatar coordinates
# avatar_local_pose_rmat = avatar_local_pose_rmat.reshape(length, -1), # length x 22 x 3 x 3 , avatar_local_rotation (all)
# avatar_position_global = avatar_position_global.reshape(length, -1), # length x 22 x 3     , avatar_global_translation (all)
        
# network inputs
# p_a = p_a.reshape(length, -1),                              # length x 3(ej) x 3 (pos)
# R_a = R_a.reshape(length, -1),                              # length x 3(ej) x 3 x 3 (rotmat)
# M = M.reshape(length, -1),                                  # length x 3 x 3(rotmat, avatar pos inverse matrix)
# aw = aw.reshape(length, -1),                                # length x 3 (avatar pos vector) *not inverse

##########################################################################################
# WE NEED Unity_trans.npy, Unity_pose_rmat.npy !!!!

###### input processing phase #####
# rotation, position data of 3 controllers
controller_rot_global = np.load(vr_data)['controller_rot_global'] # (frame, num_input*3*3)
controller_trans_global = np.load(vr_data)['controller_trans_global'] # (frame, num_input*3)

input_rotation_global_6d = utils_transform.matrot2sixd(vr_data.reshape(-1,3,3))
input_rotation_velocity_matrot = torch.matmul(torch.inverse(controller_rot_global[:-1], controller_rot_global[1:]))
input_rotation_velocity_6d = utils_transform.matrot2sixd(input_rotation_velocity_matrot.reshape(-1, 3, 3)).reshape(input_rotation_velocity_matrot.shape[0], -1, 6)
position_global_full_gt_world = controller_trans_global.reshape(controller_trans_global.shape[0], 3, 3)

num_frames = position_global_full_gt_world.shape[0]-1

hmd_position_global_full_gt_list = torch.cat([
                                                input_rotation_global_6d.reshape(num_frames,-1),
                                                input_rotation_velocity_6d.reshape(num_frames,-1),
                                                position_global_full_gt_world[1:].reshape(num_frames,-1), 
                                                position_global_full_gt_world[1:].reshape(num_frames,-1)-position_global_full_gt_world[:-1,].reshape(num_frames,-1)], 
                                                  dim=-1)


##### network eval phase#####
model.load_state_dict(pretrained_model)
model.eval()

global_orientation, joint_rotation, joint_position = model(input) # output: (num_frames, 22 * 6)


##### make pelvis trans ##### 
pose_6d = torch.cat([global_orientation, joint_rotation],dim=1).reshape(-1,6)
uniy_pose_rmat = utils_transform.sixd2matrot(pose_6d)

# Use HMD position to calculate root trans.
head_trans = controller_trans_global.reshape(controller_trans_global.shape[0], 3, 3)[:,0,:]
pred_head2root = joint_position[:,15,:]
pelv_trans = -pred_head2root+head_trans

np.savez(
        name, 
        unity_trans = pelv_trans,
        uniy_pose_rmat = uniy_pose_rmat
)

# Make Unity_trans.npy, Unity_pose_rmat.npy



# # save npz file to run vr
# np.savez('./input.npz', output)


NameError: name 'vr_data' is not defined